In [2]:
import os
import os.path
import glob
import pandas as pd
import numpy as np
from PIL import Image
import preprocessing as proc
from pandas.api.types import CategoricalDtype
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = (15, 5)
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14

pd.options.display.max_columns = 1000

DATA_PATH = '../datasets/'
TIFF_PATH = '/Users/vladarozova/Dropbox/New experiment/Images/tiff/'

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

## Load metadata

In [3]:
metadata = proc.load_data("Metadata.csv", data_path=DATA_PATH)
metadata.head()

,Stiffness,Combination,Well,Site,Channel,Fluorophore,Gain,Offset,Power,Attenuation,PCF,zPosition,zOffset,USER
0,0.2,B,A1,1,1,AF647,700,0,0.005,0.95,7679.330711,795.356,0.003333,44709390
1,0.2,B,A1,1,2,AF568,680,0,0.020,0.95,7679.330711,795.356,0.003333,44709390
2,0.2,B,A1,1,3,FITC,730,0,0.030,0.98,7679.330711,795.356,0.003333,44709390
3,0.2,B,A1,1,4,DAPI,550,0,0.024,0.98,7679.330711,795.356,0.003333,44709390
4,0.2,B,A1,2,1,AF647,700,0,0.005,0.95,7679.330711,795.272,0.003332,44709390


In [4]:
metadata.rename(str.lower, axis='columns', inplace=True)
metadata = proc.create_label(metadata, per_cell=False)
metadata.head()

,stiffness,combination,well,site,channel,fluorophore,gain,offset,power,attenuation,pcf,zposition,zoffset,user,label
0,0.2,B,A1,1,1,AF647,700,0,0.005,0.95,7679.330711,795.356,0.003333,44709390,0.2-B-A1-1
1,0.2,B,A1,1,2,AF568,680,0,0.020,0.95,7679.330711,795.356,0.003333,44709390,0.2-B-A1-1
2,0.2,B,A1,1,3,FITC,730,0,0.030,0.98,7679.330711,795.356,0.003333,44709390,0.2-B-A1-1
3,0.2,B,A1,1,4,DAPI,550,0,0.024,0.98,7679.330711,795.356,0.003333,44709390,0.2-B-A1-1
4,0.2,B,A1,2,1,AF647,700,0,0.005,0.95,7679.330711,795.272,0.003332,44709390,0.2-B-A1-2


In [9]:
norm_to = {
    'DAPI' : 600,
    'FITC' : 730,
    'AF568' : 680,
    'AF647' : 700
}
coeff = []
for ind, row in metadata.iterrows():
    coeff.append(norm_to[row.fluorophore] / row.gain)
metadata['coeff'] = coeff

In [10]:
metadata.head()

,stiffness,combination,well,site,channel,fluorophore,gain,offset,power,attenuation,pcf,zposition,zoffset,user,label,coeff
0,0.2,B,A1,1,1,AF647,700,0,0.005,0.95,7679.330711,795.356,0.003333,44709390,0.2-B-A1-1,1.000000
1,0.2,B,A1,1,2,AF568,680,0,0.020,0.95,7679.330711,795.356,0.003333,44709390,0.2-B-A1-1,1.000000
2,0.2,B,A1,1,3,FITC,730,0,0.030,0.98,7679.330711,795.356,0.003333,44709390,0.2-B-A1-1,1.000000
3,0.2,B,A1,1,4,DAPI,550,0,0.024,0.98,7679.330711,795.356,0.003333,44709390,0.2-B-A1-1,1.090909
4,0.2,B,A1,2,1,AF647,700,0,0.005,0.95,7679.330711,795.272,0.003332,44709390,0.2-B-A1-2,1.000000


In [7]:
metadata.stiffness.unique()

[0.2, 0.5, 16.0, 2.0, 32.0, 64.0, 8.0]
Categories (7, object): [0.2 < 0.5 < 2.0 < 8.0 < 16.0 < 32.0 < 64.0]

## Load images

In [8]:
# Load one image at a time
# Create image label
# Find corresponding gain730 in metadata dataframe
# Multiply all pixels by gain730, set values >= 65535 to 65535
# Convert to unsigned int
# Save image as tiff without any transformations

stiffness = ["0.2", "0.5", "2", "8", "16", "32", "64"]

for s in stiffness:
    c = 'B'
    image_path = os.path.join(TIFF_PATH, 'Cytosoft ' + s + ' kPa/Combination ' + c)
    s = str(float(s))
    os.chdir(image_path)
    filenames = glob.glob("*.tif")
    channels = [name for name in filenames if 'mask' not in name and 'corr' not in name]
    for channel in channels:
        w, i, m = channel.split('.')[0].split('-')
        if c == "B":
            f = "DAPI" * (m == "DAPI") + "FITC" * (m == "WGA") +\
            "AF568" * (m == "Vim") + "AF647" * (m == "Ker")
        label = "-".join([s, c, w, i])
        coeff = metadata.loc[(metadata.label == label) & (metadata.fluorophore == f), 'coeff'].values
        image = plt.imread(channel)
        image_corr = image * coeff
        image_corr[image_corr > 65535] = 65535
        image_corr = np.uint16(image_corr)
        im = Image.fromarray(image_corr)
        im.save("-".join([w, i, m, "corr"]) + ".tif")